In [8]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

def calculate_spearman_correlation(preprocessed_file):
    """
    Calculate Spearman correlation for all EEG channels for each patient
    and store the results in a DataFrame.
    
    Parameters:
    preprocessed_file : str
        Path to the preprocessed EEG CSV file.
    
    Returns:
    dict
        A dictionary with patient IDs as keys and their Spearman correlation DataFrames as values.
    """
    # Load the preprocessed EEG data
    df = pd.read_csv(preprocessed_file)
    
    # Group data by Patient_ID
    grouped = df.groupby('Patient_ID')
    
    # Dictionary to store Spearman correlation for each patient
    spearman_results = {}
    
    for patient_id, group in grouped:
        # Select only EEG channel columns
        eeg_columns = group.columns.difference(['Label', 'Patient_ID'])
        # Calculate Spearman correlation
        spearman_corr = group[eeg_columns].corr(method='spearman')
        # Store in the dictionary
        spearman_results[patient_id] = spearman_corr
    
    return spearman_results

def plot_spearman_heatmap(spearman_results, output_dir):
    """
    Plot heatmaps for Spearman correlation coefficients for each patient
    and save them in the specified output directory.
    
    Parameters:
    spearman_results : dict
        A dictionary with patient IDs as keys and their Spearman correlation DataFrames as values.
    output_dir : str
        The directory where heatmaps will be saved.
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    for patient_id, corr_matrix in spearman_results.items():
        plt.figure(figsize=(12, 10))  # Set figure size for the heatmap
        sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', square=True, 
                    cbar_kws={"shrink": .8}, linewidths=0.5, fmt=".2f")
        plt.title(f'Spearman Correlation Heatmap for Patient ID: {patient_id}', fontsize=16)
        plt.xlabel('EEG Channels', fontsize=14)
        plt.ylabel('EEG Channels', fontsize=14)
        plt.xticks(rotation=45)
        plt.yticks(rotation=45)
        plt.tight_layout()
        
        # Save the heatmap as an image file in the output directory
        plt.savefig(os.path.join(output_dir, f'spearman_heatmap_patient_{patient_id}.png'))
        plt.close()  # Close the figure after saving to free up memory


In [9]:
# Example usage
preprocessed_file = 'preprocessed_eeg_data.csv'  # Update this to your preprocessed CSV file path
output_dir = 'Spearman'  # Update this to your desired output directory
os.makedirs(output_dir, exist_ok=True)  # Create output directory if it doesn't exist
spearman_results = calculate_spearman_correlation(preprocessed_file)
plot_spearman_heatmap(spearman_results, output_dir)
